In [30]:
import pandas as pd
import requests
from tqdm import tqdm

## KAKAO MAP API

In [31]:
api_key_url = '../API_KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)

In [32]:
open_api_key_file['사이트'].value_counts()

사이트
공공데이터                 1
kakao_api_key_REST    1
카카오 JS                1
국토부v-world            1
GOOGLE_MAP            1
Name: count, dtype: int64

In [33]:
KAKAO_API_KEY = open_api_key_file[open_api_key_file['사이트'] == 'kakao_api_key_REST'].values[0][1]  

In [34]:
def get_lat_lon_from_address(input_address): 
    """
    카카오맵 API를 사용하여 주소를 위도, 경도로 변환하는 함수
    
    :param address: 변환할 주소 (예: "서울 광진구 능동 25")
    :param api_key: 카카오 REST API 키
    :return: (위도, 경도) 튜플 반환, 변환 실패 시 None 반환
    """
    address = "서울시 "+ input_address
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": address}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        result = response.json()
        if result["documents"]:
            x = result["documents"][0]["x"]  # 경도 (Longitude)
            y = result["documents"][0]["y"]  # 위도 (Latitude)
            return y, x 
    return None, None

In [35]:
url = "./make_file/통합_서울시_열선_정리(기점_종점).csv"

In [36]:
data = pd.read_csv(url, encoding="UTF-8")

In [37]:
data.isnull().sum()

기점_주소    0
종점_주소    0
연장(m)    0
dtype: int64

In [38]:
try :
    data = data.drop(columns='Unnamed: 0')
except KeyError as e :
    print(e)

"['Unnamed: 0'] not found in axis"


In [13]:
data.index = data.index + 1
data

,기점_주소,종점_주소,연장(m)
1,명륜길 74,명륜9길 1,160
2,행촌동 174-5,행촌동171-35,75
3,행촌동 201-172,행촌동171-120,95
4,평창동 435-2,평창동 448,150
5,평창동 66-28,평창동464-14,210
...,...,...,...
538,관악로 248,관악로30길 19,120
539,남현길 58,남부순환로256길 60,85
540,인헌16길 47,인헌16길 35,250
541,관악로28길 22,행운9길 34,100


In [14]:
data.columns

Index(['기점_주소', '종점_주소', '연장(m)'], dtype='object')

In [15]:
start_address_list = data['기점_주소'].to_list()
end_address_list = data['종점_주소'].to_list()
len_list = data['연장(m)'].to_list()

In [16]:
get_start_address_lat = []
get_start_address_lon = []

get_end_address_lat = []
get_end_address_lon = []

get_len = []

for i in tqdm(range(len(start_address_list))) : 
    search_address = start_address_list[i]
    lat, lon = get_lat_lon_from_address(search_address) 
    get_start_address_lat.append(lat)
    get_start_address_lon.append(lon) 
    get_len.append(len_list[i])
        

for i in tqdm(range(len(end_address_list))) : 
    search_address = end_address_list[i]
    lat, lon = get_lat_lon_from_address(search_address)
    get_end_address_lat.append(lat)
    get_end_address_lon.append(lon)

100%|█████████████████████████████████████████| 542/542 [00:44<00:00, 12.10it/s]


In [17]:
print(len(get_start_address_lat))
print(len(get_start_address_lon))
print(len(get_end_address_lat))
print(len(get_end_address_lon))
print(len(get_len))

542
542
542
542
542


In [18]:
df = pd.DataFrame({
    '시점_위도' : get_start_address_lat,
    '시점_경도' : get_start_address_lon,
    '종점_위도' : get_end_address_lat,
    '종점_경도' : get_end_address_lon,
    '길이' : get_len
})

In [19]:
df.isnull().sum()

시점_위도    26
시점_경도    26
종점_위도    39
종점_경도    39
길이        0
dtype: int64

In [20]:
df_not_null = df.dropna(axis=0)

In [21]:
df_not_null = df_not_null.reset_index(drop=True)

In [22]:
len(df_not_null)

485

In [23]:
df_not_null.isnull().sum()

시점_위도    0
시점_경도    0
종점_위도    0
종점_경도    0
길이       0
dtype: int64

In [29]:
try :
    save_url = "./make_file/(위경도)_서울시_도로_정리(기점_종점).csv" 
    df_not_null.to_csv(save_url, encoding ="UTF-8", index=False)
    print(f"{save_url} 파일 저장 완료")
except OSError as e :
    print(e)

./make_file/(위경도)_서울시_도로_정리(기점_종점).csv 파일 저장 완료
